<div style="background-color: #ADD8E6; border: 1px solid gray; padding: 3px">
    This notebook consists of 2 agentic workflows:
        <h3>Data Generation Workflow</h3>
        <li><b>Data Augmentation</b>: Augments the provided image dataset.</li>
        <h3>Validation Workflow</h3>
        <li><b>Image Validator</b>: Identifies whether a valid driver's license exists in the given image.</li>
        <li><b>Data Extractor</b>: Extracts relevant metadata from the image.</li>
        <li><b>Application Validator</b>: Given the extracted metadata associated with the application, uses a set of predefined rules to validate the driver's license application.</li>
</div>

In [1]:
##############################################################################
# Imports
##############################################################################
# import pysqlite3 as sqlite3
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import operator
from typing import Annotated, TypedDict, List, Optional, Literal
from langgraph.graph import StateGraph, END, START
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from PIL import Image
import pytesseract
import io
import json
from datetime import datetime
import re
import os
import requests
from io import BytesIO
from dotenv import load_dotenv
import mimetypes
import base64
from urllib.parse import urlparse
from PIL import Image
from io import BytesIO
import requests
load_dotenv()
import traceback

In [2]:
##############################################################################
# State Definitions
##############################################################################

class LicenseState(TypedDict):
    """Enhanced state definition"""
    image_path: str
    
    image_data: Optional[bytes]
    
    extracted_data: dict
    
    validation_result: dict
    
    retry_count: int
    
    messages: Annotated[List[BaseMessage], operator.add]
    
    result: dict
    
    errors: List[str]
    
    warnings: List[str]
    
    processing_time: float

In [3]:
##############################################################################
# LLMs
##############################################################################

main_llm = ChatOpenAI(
    
    model=os.getenv('LLAMASCOUT4_LLM_NAME'),
    
    api_key=os.getenv('LLAMASCOUT4_LLM_KEY'),
    
    base_url=os.getenv('LLAMASCOUT4_LLM_BASE'),

    max_tokens = 8192,

    temperature = 0
)

In [4]:
extract_data_system_prompt = """You are an expert at extracting information from U.S. driver's licenses.

Extract the following fields from the provided image. Return ONLY a valid JSON object.

Required fields:
- license_number: The driver's license number (often starts with a letter)
- first_name: First name
- middle_name: Middle name or initial (use null if not present)
- last_name: Last name/surname
- date_of_birth: Format as MM/DD/YYYY
- issue_date: License issue date (MM/DD/YYYY)
- expiration_date: License expiration date (MM/DD/YYYY)

Address fields:
- street_address: Street number and name
- city: City name
- state: Two-letter state code (e.g., CA, NY, TX)
- zip_code: 5-digit ZIP code

Physical description:
- gender: M, F, or X
- height: Format as feet'inches" (e.g., 5'10")
- weight: Weight in pounds with "lbs" (e.g., "175 lbs")
- eye_color: Three-letter code (BRN, BLU, GRN, etc.)
- hair_color: Three-letter code (BRN, BLK, BLN, etc.)

Other:
- license_class: Driver's license class (C, D, M, etc.)
- restrictions: Any restrictions (e.g., "CORRECTIVE LENSES")
- endorsements: Any endorsements
- issuing_state: Full state name

Use null for any field you cannot confidently extract.
Do not make up or guess information.

Return format:
{{
  "license_number": "value",
  "first_name": "value",
  ...
}}"""

extract_data_human_prompt = """
"Image:\n\n{image}\n\nExtract the information as JSON.
"""

image_validation_system_prompt = """
You are a data quality expert. Validate this driver's license data.

Return JSON:
{{
  "is_valid": true/false,
  "completeness_score": 0-100,
  "confidence_score": 0-100,
  "critical_issues": ["list critical problems"],
  "warnings": ["list minor issues"],
  "missing_fields": ["list missing required fields"],
  "recommendations": ["improvement suggestions"]
}}"""

image_validation_human_prompt = """You are a data quality expert. Validate this driver's license data.

Check for:
1. Critical fields present (license_number, name, date_of_birth)
2. Date format correctness and logical consistency
3. State codes are valid (2 letters)
4. ZIP codes are 5 digits
5. Gender codes are valid (M, F, X)
6. Expiration date is after issue date
7. Issue date is not in the future
8. Person is old enough to drive (check date_of_birth)

Return JSON:
{{
  "is_valid": true/false,
  "completeness_score": 0-100,
  "confidence_score": 0-100,
  "critical_issues": ["list critical problems"],
  "warnings": ["list minor issues"],
  "missing_fields": ["list missing required fields"],
  "recommendations": ["improvement suggestions"]
}}"""

In [5]:
##############################################################################
# Nodes
##############################################################################

def load_and_preprocess_image(state: LicenseState, encode_image_bytes=False) -> LicenseState:
    """Node: Load and preprocess the image"""
    print("✓ STEP 1: Image Loading & Preprocessing")
    print("="*60)

    def is_valid_http_url(input_path):
        """Returns whether or not the input is a valid URL."""

        parsed_url = urlparse(input_path)

        is_http_url = all([parsed_url.scheme in ('http', 'https'), parsed_url.netloc])

        return is_http_url
        
    
    try:
        start_time = datetime.now()

        image_path = state["image_path"]

        mime_type, _ = mimetypes.guess_type(image_path)

        if "image" in mime_type:

            if encode_image_bytes:

                if is_valid_http_url(image_path):
        
                    response = requests.get(image_path)

                    response.raise_for_status()

                    stream_to_read = response.content

                else:
                    
                    with open(image_path, "rb") as image_file:
                        
                        stream_to_read = image_file.read()
    
                img = base64.b64encode(stream_to_read).decode("utf-8")
        
                state["image_data"] = f"data:{mime_type};base64,{img}"

            else:

                state["image_data"] = image_path
                

        else:

            raise Exception(f"Mime type {mime_type} not supported")
        
        processing_time = (datetime.now() - start_time).total_seconds()
        
        print(f"✅ Image loaded: time: {processing_time:.2f}s")
        
    except Exception as e:
        
        error_msg = f"Image loading error: {str(e)}"
        
        state["errors"].append(error_msg)
        
        print(f"❌ {error_msg}")

        traceback.print_exc()
    
    return state


def extract_license_data_with_llm(state: LicenseState) -> LicenseState:
    """Node: Extract structured data"""
    print("✓ STEP 2: AI-Powered Data Extraction")
    
    print("="*60)
    
    llm = main_llm
    
    extraction_prompt = ChatPromptTemplate.from_messages([
        
        ("system", extract_data_system_prompt),
        
        ("human", extract_data_human_prompt)
    ])
    
    try:
        start_time = datetime.now()
        
        chain = extraction_prompt | llm
        
        response = chain.invoke({"image": state["image_data"]})
        
        response_text = response.content if hasattr(response, 'content') else str(response)

        print(response_text)
        
        json_text = response_text
        if "```json" in response_text:
            
            json_text = response_text.split("```json")[1].split("```")[0]
            
        elif "```" in response_text:
            
            json_text = response_text.split("```")[1].split("```")[0]
        
        extracted_data = json.loads(json_text.strip())
            
        state["extracted_data"] = extracted_data
        
        processing_time = (datetime.now() - start_time).total_seconds()

        non_null_fields = sum(1 for v in extracted_data.values() if v is not None and v != "")
        
        total_fields = len(extracted_data)
        
        print(f"✅ Extraction completed")
        
        print(f"   Fields extracted: {non_null_fields}/{total_fields}")
        
        print(f"   Time: {processing_time:.2f}s")
        
        print(f"\n   Extracted data:")
        
        print(f"   {'-'*56}")
        
        for key, value in list(extracted_data.items())[:10]:
            
            if value:
                
                display_value = str(value)[:40]
                
                print(f"   {key}: {display_value}")
                
        print(f"   {'-'*56}")
        
        state["messages"].append(AIMessage(content=response_text))
        
    except Exception as e:
        error_msg = f"Extraction error: {str(e)}"
        
        state["errors"].append(error_msg)
        
        print(f"❌ {error_msg}")
        
        state["extracted_data"] = {}
    
    return state


def validate_extracted_data(state: LicenseState) -> LicenseState:
    """Node: Validate extracted data"""
    print("✓ STEP 3: Data Validation")
    
    print("="*60)
    
    llm = main_llm
    
    validation_prompt = ChatPromptTemplate.from_messages([
        ("system", image_validation_system_prompt),
        ("human", image_validation_human_prompt)
    ])
    
    try:
        start_time = datetime.now()
        
        chain = validation_prompt | llm
        response = chain.invoke({
            "data": json.dumps(state["extracted_data"], indent=2)
        })
        
        response_text = response.content if hasattr(response, 'content') else str(response)
        
        json_text = response_text
        
        if "```json" in response_text:
            
            json_text = response_text.split("```json")[1].split("```")[0]
            
        elif "```" in response_text:
            
            json_text = response_text.split("```")[1].split("```")[0]
        
        validation_result = json.loads(json_text.strip())
        
        state["validation_result"] = validation_result
        
        processing_time = (datetime.now() - start_time).total_seconds()
        
        print(f"✅ Validation completed")
        print(f"   Valid: {validation_result.get('is_valid', False)}")
        print(f"   Completeness: {validation_result.get('completeness_score', 0):.0f}%")
        print(f"   Confidence: {validation_result.get('confidence_score', 0):.0f}%")
        print(f"   Time: {processing_time:.2f}s")
        
        if validation_result.get('critical_issues'):
            
            print(f"\n   ⚠️  Critical Issues:")
            
            for issue in validation_result['critical_issues']:
                
                print(f"      • {issue}")
        
        if validation_result.get('warnings'):
            
            print(f"\n   ⚡ Warnings:")
            
            for warning in validation_result['warnings']:
                
                print(f"      • {warning}")
        
    except Exception as e:
        error_msg = f"Validation error: {str(e)}"
        
        state["errors"].append(error_msg)
        
        print(f"❌ {error_msg}")
        
        state["validation_result"] = {
            "is_valid": False,
            "completeness_score": 0,
            "confidence_score": 0,
            "critical_issues": ["Validation process failed"],
            "warnings": [],
            "missing_fields": [],
            "recommendations": ["Manual review required"]
        }
    
    return state


def compile_final_result(state: LicenseState) -> LicenseState:
    """Node: Compile final results"""
    print("✓ STEP 4: Compiling Final Results")
    
    print("="*60)
    
    has_errors = len(state["errors"]) > 0
    
    is_valid = state["validation_result"].get("is_valid", False)
    
    completeness = state["validation_result"].get("completeness_score", 0)
    
    if has_errors:
        
        status = "error"
        
    elif not is_valid or completeness < 50:
        
        status = "needs_review"
        
    elif completeness < 80:
        
        status = "partial"
        
    else:
        
        status = "success"
    
    state["result"] = {
        
        "status": status,
        
        "extracted_data": state["extracted_data"],
        
        "validation": state["validation_result"],
        
        "metadata": {
            
            "processing_timestamp": datetime.now().isoformat(),
            
            "retry_count": state.get("retry_count", 0)
        },
        "errors": state["errors"],
        
        "warnings": state["warnings"]
    }
    
    print(f"✅ Status: {status.upper()}")
    
    print(f"   Errors: {len(state['errors'])}")
    
    print(f"   Warnings: {len(state['warnings'])}")
    
    return state

In [6]:
##############################################################################
# Graph
##############################################################################
def create_license_extraction_graph():
    """Create the LangGraph workflow"""
    workflow = StateGraph(LicenseState)
    
    # Add all nodes
    workflow.add_node("load_image", load_and_preprocess_image)
    
    workflow.add_node("extract_data", extract_license_data_with_llm)
    
    # workflow.add_node("validate_data", validate_extracted_data)
    
    workflow.add_node("compile_result", compile_final_result)
    
    # Define edges
    workflow.add_edge(START, "load_image")
    
    workflow.add_edge("load_image", "extract_data")
    
    # workflow.add_edge("extract_data", "validate_data")
    
    # workflow.add_edge("validate_data", "compile_result")

    workflow.add_edge("extract_data", "compile_result")
    
    workflow.add_edge("compile_result", END)
    
    return workflow.compile()

### Execute Code Translation Flow
Execute the flow!

In [7]:
##############################################################################
# Execute the Flow
##############################################################################
paths = [
    ("https://raw.githubusercontent.com/agapebondservant/dla_poc/refs/heads/main/notebooks/data2/DENVER-25CAP-00000-04SVL-ID.jpeg", "data2/DENVER-25CAP-00000-04SVL-ID.json")
]

def extract_license_info(image_path: str) -> dict:
        """
        Main extraction function
        """
        start_time = datetime.now()
    
        print(f"Image: {image_path}")
        print(f"Started: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

        app = create_license_extraction_graph()
        
        initial_state = {
            "image_path": image_path,
            
            "image_data": None,
            
            "extracted_data": {},
            
            "validation_result": {},
            
            "retry_count": 0,
            
            "messages": [],
            
            "result": {},
            
            "errors": [],
            
            "warnings": [],
            
            "processing_time": 0.0
        }
        
        # Run workflow
        final_state = app.invoke(initial_state)
        
        # Calculate total time
        total_time = (datetime.now() - start_time).total_seconds()
    
        final_state["result"]["metadata"]["total_processing_time"] = total_time
        
        print(f"✅ EXTRACTION COMPLETE")
    
        print(f"   Total time: {total_time:.2f}s")
    
        print("="*60 + "\n")
        
        return final_state["result"]

for image_path, application in paths:
    
    result = extract_license_info(image_path)
    
    print("📊 FINAL RESULTS")
    print("="*60)
    print(json.dumps(result, indent=2))
    
    output_file = f"license_extraction_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    with open(output_file, "w") as f:
        
        json.dump(result, f, indent=2)
    
    print(f"\n💾 Results saved to: {output_file}")

Image: https://raw.githubusercontent.com/agapebondservant/dla_poc/refs/heads/main/notebooks/data2/DENVER-25CAP-00000-04SVL-ID.jpeg
Started: 2025-11-22 07:06:35
✓ STEP 1: Image Loading & Preprocessing
✅ Image loaded: time: 0.00s
✓ STEP 2: AI-Powered Data Extraction
{'license_number': 'A123456', 'first_name': 'JOHN', 'middle_name': None, 'last_name': 'SMITH', 'date_of_birth': '01/01/1990', 'issue_date': '02/01/2020', 'expiration_date': '02/01/2025', 'street_address': '123 MAIN ST', 'city': 'DENVER', 'state': 'CO', 'zip_code': '80201', 'gender': 'M', 'height': '5\'10"', 'weight': '170 lbs', 'eye_color': 'BLU', 'hair_color': 'BLK', 'license_class': 'C', 'restrictions': None, 'endorsements': None, 'issuing_state': 'Colorado'}
✅ Extraction completed
   Fields extracted: 17/20
   Time: 0.86s

   Extracted data:
   --------------------------------------------------------
   license_number: A123456
   first_name: JOHN
   last_name: SMITH
   date_of_birth: 01/01/1990
   issue_date: 02/01/2020
  